In [1]:
import pandas as pd
import numpy as np
import random
import datetime
import os

In [2]:
os.chdir('Data')

In [3]:
user = pd.read_csv('user.csv')
group_x_course = pd.read_csv('group_x_course.csv')
group = pd.read_csv('group.csv')
student = pd.read_csv('student.csv')
user_x_course = pd.read_csv('user_x_course.csv')
course = pd.read_csv('course.csv')
student_x_hw = pd.read_csv('student_x_hw.csv')
homework = pd.read_csv('homework.csv')
course_material = pd.read_csv('course_material.csv')

In [4]:
course_material.add_date = pd.to_datetime(course_material.add_date)

In [5]:
course_material.to_csv('course_material.csv', index=False)

In [6]:
user_x_course = user_x_course.drop(user_x_course.index)
student_x_hw = student_x_hw.drop(student_x_hw.index)

In [7]:
teachers_id = user[user.user_role=='teacher']['user_id'].values

In [8]:
teacher_course_dict = {}
tcourses = set()
for tid in teachers_id:
    cid = random.choice(course.course_id.values)
    teacher_course_dict[tid] = [cid]
    tcourses.add(cid)
    for i in range(random.choice([0, 1, 2])):
        cid = random.choice(course.course_id.values)
        if cid not in teacher_course_dict[tid]:
            teacher_course_dict[tid].append(cid)
            tcourses.add(cid)

In [9]:
if 1 not in teacher_course_dict[tid]:
    teacher_course_dict[tid].append(1)
    tcourses.add(1)

In [10]:
teacher_course_dict

{3: [2, 4], 4: [3, 4], 16: [2, 1]}

In [11]:
uids = []
courses = []
roles = []
for k, vals in teacher_course_dict.items():
    for v in vals:
        uids.append(k)
        courses.append(v)
        roles.append('teacher')

In [12]:
student_id = user[(user.user_role=='student') & (~user['password'].isna())].sample(len(student)//5).user_id.values

In [13]:
student_course_dict = {}
for sid in student_id:
    gid = student[student.user_id == sid].group_id.values[0]
    student_course_dict[sid] = group_x_course[group_x_course.group_id==gid].course_id.values

In [14]:
for k, vals in student_course_dict.items():
    for v in vals:
        if (v in tcourses) and random.choice([True, False]):
            uids.append(k)
            courses.append(v)
            roles.append('confidant')

In [15]:
user_x_course['user_id'] = uids
user_x_course['course_id'] = courses
user_x_course['course_role'] = roles

In [16]:
user_x_course

,user_id,course_id,course_role
0,3,2,teacher
1,3,4,teacher
2,4,3,teacher
3,4,4,teacher
4,16,2,teacher
5,16,1,teacher
6,14,1,confidant
7,14,2,confidant
8,14,3,confidant
9,15,3,confidant


In [17]:
user_x_course.to_csv('user_x_course.csv', index=False)

In [18]:
studenthw_id = user[(user.user_role=='student') & (~user['password'].isna())].user_id.values

In [19]:
studenthw_course_dict = {}
for sid in studenthw_id:
    gid = student[student.user_id == sid].group_id.values[0]
    if 1 in group_x_course[group_x_course.group_id==gid].course_id.values:
        studenthw_course_dict[sid] = group_x_course[group_x_course.group_id==gid].course_id.values

In [20]:
studenthw_course_dict

{14: array([1, 2, 3]), 15: array([1, 2, 3])}

In [21]:
homework.time_start = pd.to_datetime(homework.time_start)
homework.time_end = pd.to_datetime(homework.time_end)

In [22]:
days = (homework.time_end[0] - homework.time_start[0]).days

In [23]:
homework.time_start[0] + datetime.timedelta(days=19) + datetime.timedelta(hours=23)

Timestamp('2020-10-21 11:00:00')

In [24]:
hw_idx = homework.hw_id.values

In [25]:
hwusers = []
hw_id = []
time_send = []
for user in studenthw_course_dict:
    for hwid in hw_idx:
        hwusers.append(user)
        hw_id.append(hwid)
        time_send.append(homework[homework.hw_id==hwid].time_start[homework[homework.hw_id==hwid].time_start.index[0]] \
                         + datetime.timedelta(days=int(random.choice(np.arange(days)))) \
                         + datetime.timedelta(hours=int(random.choice(np.arange(24)))))

In [26]:
student_x_hw['user_id'] = hwusers

In [27]:
student_x_hw['hw_id'] = hw_id

In [28]:
student_x_hw['time_send'] = time_send

In [29]:
student_x_hw['solution'] = 'some code'

In [30]:
student_x_hw.to_csv('student_x_hw.csv', index=False)

In [31]:
homework.to_csv('homework.csv', index=False)